In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp
import excel_formating

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca.dropna(axis=0, subset=['Account'], inplace=True)
ca.drop(list(ca[ca['Account']==0].index), axis=0, inplace=True)
ca.dropna(axis=0, subset=['Request ID'], inplace=True)

In [ ]:
m = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/coverage_mails.xlsx"\
    ,filename = 'coverage_mails.xlsx')
mail = pd.read_excel('coverage_mails.xlsx')

In [ ]:
#SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h1>Hola</h1>
        <h3>Por favor revisa el archivo adjunto para encontrar el detalle de los errores</h3>
        <h3>Asegurate de que tus registros esten corregidos y al d&iacute;a</h3>"""

In [ ]:


def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if pd.isnull(value) or value != value:
                #Fechas vacias
                value = datetime.strptime('2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                #Error de formato
                value = datetime.strptime('2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            #Fechas normales
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    


In [ ]:
# Request Type

request_type = ca[['Request ID', 'Request Type']].copy()

for i, row in request_type.iterrows():
    if pd.isnull(row['Request Type']) == True:
        request_type['Request Type'].loc[i] = 'Request Type Empty'
    else:
        request_type['Request Type'].loc[i] = None

request_type.dropna(axis=0, subset=['Request Type'], inplace=True)


In [ ]:
# Join Call

join_call = ca[['Request ID', 'Join Call']].copy()

for i, row in join_call.iterrows():
    if pd.isnull(row['Join Call']) == True:
        join_call['Join Call'].loc[i] = 'Join Call Empty'
    else:
        join_call['Join Call'].loc[i] = None

join_call.dropna(axis=0, subset=['Join Call'], inplace=True)


In [ ]:
# Upcoming Renewal
upcoming_renewal = ca[['Request ID', 'Upcoming Renewal']].copy()

for index, row in upcoming_renewal.iterrows():
    if pd.isna(row['Upcoming Renewal']) == True:
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Empty Value'
    elif len(row['Upcoming Renewal']) > 6:
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Value with space'
    else:    
        upcoming_renewal['Upcoming Renewal'].loc[index] = None

upcoming_renewal.dropna(axis=0, subset=['Upcoming Renewal'], inplace=True)


In [ ]:
# Date Received

date_received = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
date_received['Date received'] = format_date_column(date_received['Date received'])
date_received['PAR Requested'] = format_date_column(date_received['PAR Requested'])

for index, row in date_received.iterrows():
    if row['Date received'] == Timestamp('2030-2-2 00:00:00'):
        date_received['Date received'].loc[index] = 'Empty Date'
    elif row['Date received'] == Timestamp('2050-2-2 00:00:00'):
        date_received['Date received'].loc[index] = 'Format Error'
    elif row['Date received'] < Timestamp('2021-10-15 00:00:00'):
        date_received['Date received'].loc[index] = '15/oct error'
    elif row['Date received'] > row['PAR Requested']:
        if row['PAR Requested'] != Timestamp('2030-2-2 00:00:00') and row['PAR Requested'] != Timestamp('2050-2-2 00:00:00'):
            date_received['Date received'].loc[index] = 'Date greater than PAR requested'
        else:
            date_received['Date received'].loc[index] = None
    else:
        date_received['Date received'].loc[index] = None

date_received.dropna(axis=0, subset=['Date received'], inplace=True)
date_received.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
# Coverage Advisor

cov_adv = ca[['Request ID', 'Coverage Advisor']].copy()

for index, row in cov_adv.iterrows():
    if pd.isnull(row['Coverage Advisor']) == True:
        cov_adv['Coverage Advisor'].loc[index] = 'Empty Name'
    elif set(row['Coverage Advisor']).difference(printable):
        cov_adv['Coverage Advisor'].loc[index] = 'Special Caracter in Name'
    else:
        cov_adv['Coverage Advisor'].loc[index] = None

cov_adv.dropna(axis=0, subset=['Coverage Advisor'], inplace=True)

In [ ]:
# PAR Requested

PAR_req = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
PAR_req['Date received'] = format_date_column(PAR_req['Date received'])
PAR_req['PAR Requested'] = format_date_column(PAR_req['PAR Requested'])

for index, row in PAR_req.iterrows():
    if row['PAR Requested'] == Timestamp('2030-2-2 00:00:00'):
        PAR_req['PAR Requested'].loc[index] = 'Empty Date'
    elif row['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
        PAR_req['PAR Requested'].loc[index] = 'Format Error'
    elif row['Date received'] > row['PAR Requested']:
        if row['Date received'] != Timestamp('2030-2-2 00:00:00') and row['Date received'] != Timestamp('2050-2-2 00:00:00'):
            PAR_req['PAR Requested'].loc[index] = 'Date Received greater than PAR requested'
        else:
            PAR_req['PAR Requested'].loc[index] = None
    else:
        PAR_req['PAR Requested'].loc[index] = None

PAR_req.dropna(axis=0, subset=['PAR Requested'], inplace=True)
PAR_req.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# Date 1st contact
d_f_contact = ca[['Request ID', 'Date received', 'Date First Contact']].copy()
d_f_contact['Date received'] = format_date_column(d_f_contact['Date received'])
d_f_contact['Date First Contact'] = format_date_column(d_f_contact['Date First Contact'])

for index, row in d_f_contact.iterrows():
    if row['Date First Contact'] == Timestamp('2030-2-2 00:00:00'):
        d_f_contact['Date First Contact'].loc[index] = None
    elif row['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
        d_f_contact['Date First Contact'].loc[index] = 'Format Error'
    elif row['Date received'] > row['Date First Contact']:
        if row['Date received'] != Timestamp('2030-2-2 00:00:00') and row['Date received'] != Timestamp('2050-2-2 00:00:00'):
            d_f_contact['Date First Contact'].loc[index] = 'Date Received greater than First Contact'
        else:
            d_f_contact['Date First Contact'].loc[index] = None
    else:
        d_f_contact['Date First Contact'].loc[index] = None

d_f_contact.dropna(axis=0, subset=['Date First Contact'], inplace=True)
d_f_contact.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# PAR Received

PAR_recev = ca[['Request ID', 'PAR Requested', 'PAR Received']].copy()
PAR_recev['PAR Requested'] = format_date_column(PAR_recev['PAR Requested'])
PAR_recev['PAR Received'] = format_date_column(PAR_recev['PAR Received'])

for index, row in PAR_recev.iterrows():
    if row['PAR Received'] == Timestamp('2030-2-2 00:00:00'):
        PAR_recev['PAR Received'].loc[index] = None
    elif row['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
        PAR_recev['PAR Received'].loc[index] = 'Format Error'
    elif row['PAR Requested'] > row['PAR Received']:
        if row['PAR Requested'] != Timestamp('2030-2-2 00:00:00') and row['PAR Requested'] != Timestamp('2050-2-2 00:00:00'):
            PAR_recev['PAR Received'].loc[index] = 'PAR Requested greater than PAR received'
        else:
            PAR_recev['PAR Received'].loc[index] = None
    else:
        PAR_recev['PAR Received'].loc[index] = None

PAR_recev.dropna(axis=0, subset=['PAR Received'], inplace=True)
PAR_recev.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
# Date Analysis

dat_nalys = ca[['Request ID', 'PAR Received', 'Date Analysis']].copy()
dat_nalys['Date Analysis'] = format_date_column(dat_nalys['Date Analysis'])
dat_nalys['PAR Received'] = format_date_column(dat_nalys['PAR Received'])

for index, row in dat_nalys.iterrows():
    if row['Date Analysis'] == Timestamp('2030-2-2 00:00:00'):
        dat_nalys['Date Analysis'].loc[index] = None
    elif row['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
        dat_nalys['Date Analysis'].loc[index] = 'Format Error'
    elif row['PAR Received'] > row['Date Analysis']:
        if row['PAR Received'] != Timestamp('2030-2-2 00:00:00') and row['PAR Received'] != Timestamp('2050-2-2 00:00:00'):
            dat_nalys['Date Analysis'].loc[index] = 'PAR received greater than Date Analysis'
        else:
            dat_nalys['Date Analysis'].loc[index] = None
    else:
        dat_nalys['Date Analysis'].loc[index] = None

dat_nalys.dropna(axis=0, subset=['Date Analysis'], inplace=True)
dat_nalys.drop(axis=1, columns=['PAR Received'], inplace=True)

In [ ]:
# Pre Readout Effort

pre_reffor = ca[['Request ID', 'Pre-Readout Effort (min)', 'Account Stage']].copy()

for j, row in pre_reffor.iterrows():
    if pd.isna(row['Pre-Readout Effort (min)']) == True:
        if pd.isna(row['Account Stage']) == False:
            if row['Account Stage'][0] == 'F':
                pre_reffor['Pre-Readout Effort (min)'].loc[j] = 'empty value'
    elif row['Pre-Readout Effort (min)'] > 180:
        pre_reffor['Pre-Readout Effort (min)'].loc[j] = 'value above 180'
    else:
        pre_reffor['Pre-Readout Effort (min)'].loc[j] = None

pre_reffor.dropna(axis=0, subset=['Pre-Readout Effort (min)'], inplace=True)
pre_reffor.drop(axis=1, columns=['Account Stage'], inplace=True)

In [ ]:
# Date Inite Sent

date_init_s = ca[['Request ID', 'Date First Contact', 'Date Invite Sent']].copy()
date_init_s['Date Invite Sent'] = format_date_column(date_init_s['Date Invite Sent'])
date_init_s['Date First Contact'] = format_date_column(date_init_s['Date First Contact'])

for index, row in date_init_s.iterrows():
    if row['Date Invite Sent'] == Timestamp('2030-2-2 00:00:00'):
        date_init_s['Date Invite Sent'].loc[index] = None
    elif row['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
        date_init_s['Date Invite Sent'].loc[index] = 'Format Error'
    elif row['Date First Contact'] > row['Date Invite Sent']:
        if row['Date First Contact'] != Timestamp('2030-2-2 00:00:00') and  row['Date First Contact'] != Timestamp('2050-2-2 00:00:00'):
            date_init_s['Date Invite Sent'].loc[index] = 'Date First Contact greater than Date Invite Sent'
        else:
            date_init_s['Date Invite Sent'].loc[index] = None
    else:
        date_init_s['Date Invite Sent'].loc[index] = None

date_init_s.dropna(axis=0, subset=['Date Invite Sent'], inplace=True)
date_init_s.drop(axis=1, columns=['Date First Contact'], inplace=True)

In [ ]:
# Date Schedule

date_sche = ca[['Request ID', 'Date Invite Sent', 'Date Scheduled', 'Date Readout Done']].copy()
date_sche['Date Invite Sent'] = format_date_column(date_sche['Date Invite Sent'])
date_sche['Date Scheduled'] = format_date_column(date_sche['Date Scheduled'])
date_sche['Date Readout Done'] = format_date_column(date_sche['Date Readout Done'])

for index, row in date_sche.iterrows():
    if row['Date Scheduled'] == Timestamp('2030-2-2 00:00:00'):
        date_sche['Date Scheduled'].loc[index] = None
    elif row['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        date_sche['Date Scheduled'].loc[index] = 'Format Error'
    elif row['Date Scheduled'] != row['Date Readout Done']:
        if row['Date Readout Done'] != Timestamp('2030-2-2 00:00:00') and  row['Date Readout Done'] != Timestamp('2050-2-2 00:00:00'):
            date_sche['Date Scheduled'].loc[index] = 'Scheduled and Readout Done are different'
        else:
            date_sche['Date Scheduled'].loc[index] = None
    elif row['Date Invite Sent'] > row['Date Scheduled']:
        if row['Date Invite Sent'] != Timestamp('2030-2-2 00:00:00') and  row['Date Invite Sent'] != Timestamp('2050-2-2 00:00:00'):
            date_sche['Date Scheduled'].loc[index] = 'Invite Sent greater than Scheduled'
        else:
            date_sche['Date Scheduled'].loc[index] = None
    else:
        date_sche['Date Scheduled'].loc[index] = None

date_sche.dropna(axis=0, subset=['Date Scheduled'], inplace=True)
date_sche.drop(axis=1, columns=['Date Invite Sent'], inplace=True)
date_sche.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Date Readout Done

date_red_don = ca[['Request ID', 'Date Scheduled', 'Date Readout Done', 'Status']].copy()
date_red_don['Date Scheduled'] = format_date_column(date_red_don['Date Scheduled'])
date_red_don['Date Readout Done'] = format_date_column(date_red_don['Date Readout Done'])

for index, row in date_red_don.iterrows():
    if row['Date Readout Done'] == Timestamp('2030-2-2 00:00:00'):
        if pd.isnull(row['Status']) == False:
            if row['Status'][0] != 'F':
                date_red_don['Date Readout Done'].loc[index] = 'Value mustnt be empty'
            else:
                date_red_don['Date Readout Done'].loc[index] = None
        else:
            date_red_don['Date Readout Done'].loc[index] = None
    elif row['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        date_red_don['Date Readout Done'].loc[index] = 'Format Error'
    elif row['Date Scheduled'] != row['Date Readout Done']:
        if row['Date Scheduled'] != Timestamp('2030-2-2 00:00:00') and row['Date Scheduled'] != Timestamp('2050-2-2 00:00:00'):
            date_red_don['Date Readout Done'].loc[index] = 'Scheduled and Readout Done are different'
        else:
            date_red_don['Date Readout Done'].loc[index] = None
    else:
        date_red_don['Date Readout Done'].loc[index] = None

date_red_don.dropna(axis=0, subset=['Date Readout Done'], inplace=True)
date_red_don.drop(axis=1, columns=['Date Scheduled'], inplace=True)
date_red_don.drop(axis=1, columns=['Status'], inplace=True)

In [ ]:
# Readout Duration (min)

read_min = ca[['Request ID', 'Date Readout Done', 'Readout Duration (min)']].copy()

for j, row in read_min.iterrows():
    if pd.isnull(row['Date Readout Done']) == False and pd.isnull(row['Readout Duration (min)']) == True:
        read_min['Readout Duration (min)'].loc[j] = 'Value mustnt be empty'
    elif row['Readout Duration (min)'] > 180:
        read_min['Readout Duration (min)'].loc[j] = 'Value to high'
    else:
        read_min['Readout Duration (min)'].loc[j] = None

read_min.dropna(axis=0, subset=['Readout Duration (min)'], inplace=True)
read_min.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Status

c_status = ca[['Request ID', 'Account Stage', 'Status']].copy()

for j, row in c_status.iterrows():
    if pd.isnull(row['Account Stage']) == False and pd.isnull(row['Status']) == True:
        c_status['Status'].loc[j] = 'must fill this cell'
    else:
        c_status['Status'].loc[j] = None

c_status.dropna(axis=0, subset=['Status'], inplace=True)
c_status.drop(axis=1, columns=['Account Stage'], inplace=True)

In [ ]:
# Delivery Type

del_type = ca[['Request ID', 'Date Readout Done', 'Delivery Type']].copy()

for id, row in del_type.iterrows():
    if pd.isnull(row['Delivery Type']) == True:
        del_type['Delivery Type'].loc[id] = 'Empty value'
    elif pd.isna(row['Date Readout Done']) == True and row['Delivery Type'] != 'Not presented':
        del_type['Delivery Type'].loc[id] = 'should be: Not Presented'
    else:
        del_type['Delivery Type'].loc[id] = None

del_type.dropna(axis=0, subset=['Delivery Type'], inplace=True)
del_type.drop(axis=1, columns=['Date Readout Done'], inplace=True)    

In [ ]:
# Reason Declined

reason_decl = ca[['Request ID', 'Status', 'Reason Declined']].copy()

for index, row in reason_decl.iterrows():
    if pd.isnull(row['Status']) == True:
        reason_decl['Reason Declined'].loc[index] = None
    elif row['Status'][2] == 'D' and pd.isnull(row['Reason Declined']) == True:
        reason_decl['Reason Declined'].loc[index] = 'Empty Value'
    else:
        reason_decl['Reason Declined'].loc[index] = None

reason_decl.dropna(axis=0, subset=['Reason Declined'], inplace=True)
reason_decl.drop(axis=1, columns=['Status'], inplace=True) 


In [ ]:
# Reason Declined Lvl 2

reason_decl2 = ca[['Request ID', 'Reason Declined', 'Reason Declined Lvl 2']].copy()

for index, row in reason_decl2.iterrows():
    if pd.isnull(row['Reason Declined']) == False and pd.isnull(row['Reason Declined Lvl 2']) == True:
        reason_decl2['Reason Declined Lvl 2'].loc[index] = 'Empty Value'
    else:
        reason_decl2['Reason Declined Lvl 2'].loc[index] = None

reason_decl2.dropna(axis=0, subset=['Reason Declined Lvl 2'], inplace=True)
reason_decl2.drop(axis=1, columns=['Reason Declined'], inplace=True) 

In [ ]:
# Reason On Hold and Days on Hold

reas_hold = ca[['Request ID', 'Reason On Hold', 'Days on Hold']].copy()

for index, row in reas_hold.iterrows():
    if pd.isna(row['Reason On Hold']) == True and pd.isna(row['Days on Hold']) == False:
        reas_hold['Reason On Hold'].loc[index] = 'Empty Value'
        reas_hold['Days on Hold'].loc[index] = None
    elif pd.isna(row['Reason On Hold']) == False and pd.isna(row['Days on Hold']) == True:
        reas_hold['Days on Hold'].loc[index] = 'Empty Value'
        reas_hold['Reason On Hold'].loc[index] = None
    else:
        reas_hold['Reason On Hold'].loc[index] = None
        reas_hold['Days on Hold'].loc[index] = None

reas_hold.dropna(axis=0, how='all', subset=['Reason On Hold', 'Days on Hold'], inplace=True) 

In [ ]:
# Miss Attach Opportunity // Expired Opportunity // LDoS Opportunity

oppty= ca[['Request ID', 'Account Stage','Miss Attach Opportunity', 'Expired Opportunity', 'LDoS Opportunity']].copy()

for p, row in oppty.iterrows():
    if pd.isna(row['Account Stage']) == False:
        if pd.isna(row['Miss Attach Opportunity']) == True:
            oppty['Miss Attach Opportunity'].loc[p] = 'Empty oppotunity'
        if pd.isna(row['Expired Opportunity']) == True:
            oppty['Expired Opportunity'].loc[p] = 'Empty oppotunity'
        if pd.isna(row['LDoS Opportunity']) == True:
            oppty['LDoS Opportunity'].loc[p] = 'Empty oppotunity'

for index, row in oppty[oppty.columns[2:]].iterrows():
    for loc, sheet in row.items():
        if sheet != 'Empty oppotunity':
            oppty[loc].loc[index] = None

oppty.dropna(axis=0, how='all', subset=['Miss Attach Opportunity', 'Expired Opportunity', 'LDoS Opportunity'], inplace=True)
oppty.drop(axis=1, columns=['Account Stage'], inplace=True)


In [ ]:
# Follow up 1

follow1 = ca[['Request ID', 'Date Readout Done','Date Follow Up 1']].copy()
follow1['Date Readout Done'] = format_date_column(follow1['Date Readout Done'])
follow1['Date Follow Up 1'] = format_date_column(follow1['Date Follow Up 1'])

for index, row in follow1.iterrows():
    if row['Date Readout Done'] != Timestamp('2030-2-2 00:00:00') and row['Date Readout Done'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Follow Up 1'] != Timestamp('2030-2-2 00:00:00') and  row['Date Follow Up 1'] != Timestamp('2050-2-2 00:00:00'):
            if row['Date Readout Done'] > row['Date Follow Up 1']:
                follow1['Date Follow Up 1'].loc[index] = 'Readout greater than follow up'
        if row['Date Follow Up 1'] == Timestamp('2030-2-2 00:00:00'):
            follow1['Date Follow Up 1'].loc[index] = 'Empty Date'
        if row['Date Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
            follow1['Date Follow Up 1'].loc[index] = 'Format Error'
    
for index, row in follow1.iterrows():
   if type(row['Date Follow Up 1']) == Timestamp:
      follow1['Date Follow Up 1'].loc[index] = None
            
follow1.dropna(axis=0, subset=['Date Follow Up 1'], inplace=True)
follow1.drop(axis=1, columns=['Date Readout Done'], inplace=True)


In [ ]:
# Follow up 2

follow2 = ca[['Request ID', 'Date Follow Up 1', 'Date Follow Up 2','Closed Date']].copy()
follow2['Closed Date'] = format_date_column(follow2['Closed Date'])
follow2['Date Follow Up 1'] = format_date_column(follow2['Date Follow Up 1'])
follow2['Date Follow Up 2'] = format_date_column(follow2['Date Follow Up 2'])

for index, row in follow2.iterrows():
    if row['Closed Date'] == Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 2'] == Timestamp('2030-2-2 00:00:00'):
        follow2['Date Follow Up 2'].loc[index] = 'should not be empty'
    if row['Date Follow Up 1'] != Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 1'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Follow Up 2'] != Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 2'] != Timestamp('2050-2-2 00:00:00'): 
            if row['Date Follow Up 1'] > row['Date Follow Up 2']:
                follow2['Date Follow Up 2'].loc[index] = 'Follow 1 greater'
    if row['Date Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
            follow2['Date Follow Up 2'].loc[index] = 'Format Error'
    
for index, row in follow2.iterrows():
   if type(row['Date Follow Up 2']) == Timestamp:
      follow2['Date Follow Up 2'].loc[index] = None
            
follow2.dropna(axis=0, subset=['Date Follow Up 2'], inplace=True)
follow2.drop(axis=1, columns=['Date Follow Up 1'], inplace=True)
follow2.drop(axis=1, columns=['Closed Date'], inplace=True)

In [ ]:
# Follow up 3

follow3 = ca[['Request ID', 'Date Follow Up 2', 'Date Follow Up 3','Closed Date']].copy()
follow3['Closed Date'] = format_date_column(follow3['Closed Date'])
follow3['Date Follow Up 2'] = format_date_column(follow3['Date Follow Up 2'])
follow3['Date Follow Up 3'] = format_date_column(follow3['Date Follow Up 3'])

for index, row in follow3.iterrows():
    if row['Closed Date'] == Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 3'] == Timestamp('2030-2-2 00:00:00'):
        follow3['Date Follow Up 3'].loc[index] = 'should not be empty'
    if row['Date Follow Up 2'] != Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 2'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Follow Up 3'] != Timestamp('2030-2-2 00:00:00') and row['Date Follow Up 3'] != Timestamp('2050-2-2 00:00:00'): 
            if row['Date Follow Up 2'] > row['Date Follow Up 3']:
                follow3['Date Follow Up 3'].loc[index] = 'Follow 2 greater'
    if row['Date Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
            follow3['Date Follow Up 3'].loc[index] = 'Format Error'
    
for index, row in follow3.iterrows():
   if type(row['Date Follow Up 3']) == Timestamp:
      follow3['Date Follow Up 3'].loc[index] = None
            
follow3.dropna(axis=0, subset=['Date Follow Up 3'], inplace=True)
follow3.drop(axis=1, columns=['Date Follow Up 2'], inplace=True)
follow3.drop(axis=1, columns=['Closed Date'], inplace=True)

In [ ]:
# Pre Readout Effort

follow_eft = ca[['Request ID', 'Date Readout Done','Follow Up Effort (min)']].copy()

for f, row in follow_eft.iterrows():
    if pd.isnull(row['Date Readout Done']) == False and pd.isnull(row['Follow Up Effort (min)']) == True:
        follow_eft['Follow Up Effort (min)'].loc[f] = 'Should not be empty'
    elif pd.isnull(row['Follow Up Effort (min)']) == False:
        if type(row['Follow Up Effort (min)']) == int or type(row['Follow Up Effort (min)']) == float:
            if row['Follow Up Effort (min)'] > 240:
               follow_eft['Follow Up Effort (min)'].loc[f] = 'Value too hight'
            else:
                follow_eft['Follow Up Effort (min)'].loc[f] = None
        else:
            follow_eft['Follow Up Effort (min)'].loc[f] = None
    else:
        follow_eft['Follow Up Effort (min)'].loc[f] = None

follow_eft.dropna(axis=0, subset=['Follow Up Effort (min)'], inplace=True)
follow_eft.drop(axis=1, columns=['Date Readout Done'], inplace=True)


In [ ]:
# Closing Date

closig_dat = ca[['Request ID', 'Date received','Status', 'Closed Date']].copy()
closig_dat['Closed Date'] = format_date_column(closig_dat['Closed Date'])
closig_dat['Date received'] = format_date_column(closig_dat['Date received'])

for i, row in closig_dat.iterrows():

    if row['Closed Date'] != Timestamp('2030-2-2 00:00:00') and row['Closed Date'] != Timestamp('2050-2-2 00:00:00'): 
        if row['Date received'] != Timestamp('2030-2-2 00:00:00') and row['Date received'] != Timestamp('2050-2-2 00:00:00'):
            if row['Date received'] > row['Closed Date']:
                closig_dat['Closed Date'].loc[i] = 'Date Received greater'
    elif row['Closed Date'] == Timestamp('2050-2-2 00:00:00'):
        closig_dat['Closed Date'].loc[i] = 'Format Error'
    elif row['Closed Date'] == Timestamp('2030-2-2 00:00:00'):
        if pd.isnull(row['Status']) == False:
            if row['Status'][5] != 'H' and row['Status'][2] != 'S' and row['Status'][2] != 'R':
                if row['Status'][2] == 'D' or row['Status'][2] == 'U' or (row['Status'][2] == 'O' and row['Status'][14] == 'L'):
                    closig_dat['Closed Date'].loc[i] = 'Empty Value'


for index, row in closig_dat.iterrows():
   if type(row['Closed Date']) == Timestamp:
      closig_dat['Closed Date'].loc[index] = None

closig_dat.dropna(axis=0, subset=['Closed Date'], inplace=True)
closig_dat.drop(axis=1, columns=['Date received'], inplace=True)
closig_dat.drop(axis=1, columns=['Status'], inplace=True)

### Merging tables

In [ ]:
# Merging new tables
new_table = request_type.merge(join_call, how='outer', left_on='Request ID', right_on='Request ID').merge(
            upcoming_renewal, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_received, how='outer', left_on='Request ID', right_on='Request ID').merge(
            cov_adv, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_req, how='outer', left_on='Request ID', right_on='Request ID').merge(
            d_f_contact, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_recev, how='outer', left_on='Request ID', right_on='Request ID').merge(
            dat_nalys, how='outer', left_on='Request ID', right_on='Request ID').merge(
            pre_reffor, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_init_s, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_sche, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_red_don, how='outer', left_on='Request ID', right_on='Request ID').merge(
            read_min, how='outer', left_on='Request ID', right_on='Request ID').merge(
            c_status, how='outer', left_on='Request ID', right_on='Request ID').merge(
            del_type, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl2, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reas_hold, how='outer', left_on='Request ID', right_on='Request ID').merge(
            oppty, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow1, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow2, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow3, how='outer', left_on='Request ID', right_on='Request ID').merge(
            follow_eft, how='outer', left_on='Request ID', right_on='Request ID').merge(
            closig_dat, how='outer', left_on='Request ID', right_on='Request ID')


In [ ]:
new_table.drop_duplicates(subset=['Request ID'], inplace=True)
new_table.dropna(axis=0, how='all', subset=new_table.columns[2:], inplace=True)  
t_names = ca[['Request ID','Coverage Advisor']]
new_table = t_names.merge(new_table, how='right', left_on='Request ID', right_on='Request ID')
new_table.to_excel('error_report_%s.xlsx' % today, index=False)    
excel_formating.format_excel()

In [ ]:
mail_table = new_table['Coverage Advisor_x'].to_frame().drop_duplicates().reset_index(drop=True)
mail_table = mail_table.merge(mail, how='left', left_on='Coverage Advisor_x', right_on='Coverage Advisor')
mail_table.dropna(axis=0, subset=['Email'], inplace=True)

### Send Mail

In [ ]:
#outlook server connection
outlServer = smtplib.SMTP(host='smtp-mail.outlook.com', port=587)

#Initialize encryption
outlServer.starttls()

#Login in the outlook account using the define variables username and password
outlServer.login(myMail, password)

In [ ]:
#Definition of the function to send emails
def send_msg(To, From, Subject, Message, s_attach):
    msg = MIMEMultipart()
    msg['From'] = From
    msg['To'] = To
    msg['Subject'] = Subject
    msg_content = MIMEText(Message, 'html', 'utf-8')
    msg.attach(msg_content)
       
    if s_attach == True:
        with open('error_report_%s.xlsx' % today, 'rb') as e:
            exc = MIMEBase('application', 'octet-stream', filename='error_report_%s.xlsx' % today)
            exc.add_header('content-Disposition', 'attachment', filename='error_report_%s.xlsx' % today)
            exc.set_payload(e.read())
            encoders.encode_base64(exc)
            msg.attach(exc)
    
    outlServer.sendmail(From, To, msg.as_string())

In [ ]:
for i, sheet in mail_table.iterrows():
    if pd.isnull(sheet['Email']) == False:
        send_msg(sheet['Email'], myMail, subjectEmail , msg1, True)
        print(sheet['Email'])

In [ ]:
outlServer.quit()